# Home Automation Simulation - Interactive Smart Home Control

## Use Case
This simulation targets users exploring smart home automation—homeowners, tech enthusiasts, or students—allowing them to control a virtual home and monitor resource usage via chat.

## Problem
Testing real home automation requires expensive hardware and setup. Users lack a simple, interactive platform to simulate controlling devices (lights, HVAC, locks) and tracking energy/water usage.

## Solution
A Jupyter notebook with an LLM-powered chat interface to:
- Control simulated devices (lights, security, TVs, HVAC, locks).
- Track energy (kWh) and water (gal) usage.
- Update a virtual home state based on user commands.
- Provide feedback on actions and current state, including status queries.

## How to Use
1. Run in Jupyter/Kaggle with `langchain-google-genai` installed.
2. Enter commands (e.g., "Turn on kitchen lights") or queries (e.g., "Are all lights off?") in the chat.
3. View responses and updated home state (e.g., "Lights on, Energy: 0.1 kWh" or "Yes, all lights are off").
4. Type "exit" for a final usage summary.

## Setup

In [ ]:
# Install required packages
# !pip uninstall -y langchain-google-genai  # Remove existing version to ensure clean install
!pip install -qU langchain-google-genai==2.1.2  # Reinstall to avoid corruption

## Implementation

In [ ]:
# Import necessary libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.caches import BaseCache  # Explicitly import BaseCache to resolve Pydantic issue
from langchain_core.callbacks import Callbacks  # Explicitly import Callbacks to resolve Pydantic issue
from langchain_core.messages import HumanMessage, SystemMessage
from kaggle_secrets import UserSecretsClient
import json

# Configuration
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
print("Defining ChatGoogleGenerativeAI with BaseCache and Callbacks imported...")
ChatGoogleGenerativeAI.model_rebuild()  # Rebuild model with BaseCache and Callbacks defined
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.2, max_tokens=1000)
print("LLM initialized successfully")  # Debug to confirm initialization

# Core Function: Initializes the simulated home state
# Purpose: Sets up a virtual home with rooms, floors, devices, and usage metrics
# Usage: Called at startup to create the initial home_state dictionary
def initialize_home_state() -> dict:
    return {
        "rooms": {
            "living_room": {"lights": "off", "tv": "off", "lock": "locked"},
            "kitchen": {"lights": "off", "electronics": "off", "lock": "locked"},
            "bedroom_1st": {"lights": "off", "tv": "off", "lock": "locked"},
            "bedroom_2nd": {"lights": "off", "tv": "off", "lock": "locked"}
        },
        "floors": {
            "1st": {"temp": 70, "windows": "locked"},
            "2nd": {"temp": 70, "windows": "locked"}
        },
        "security": {"status": "off", "passcode": "1234"},
        "usage": {"energy": 0.0, "water": 0.0}  # kWh, gallons
    }

# Core Function: Updates home state based on LLM actions
# Purpose: Applies changes (e.g., lights on, temp set) for single or multiple actions and tracks usage
# Usage: Called after LLM interprets a command to modify home_state; handles both single dict and "actions" array
def update_home_state(home_state: dict, action_data: dict) -> dict:
    # Handle single action or multiple actions in an "actions" array
    actions = action_data.get("actions", [action_data]) if "actions" in action_data else [action_data]
    
    for action in actions:
        if "device" not in action:
            continue
        
        device = action["device"]
        room_or_floor = action.get("location", "")
        value = action["value"]
        
        if device == "lights":
            if room_or_floor == "all":
                for room in home_state["rooms"]:
                    home_state["rooms"][room]["lights"] = value
                    if value == "on":
                        home_state["usage"]["energy"] += 0.1  # 0.1 kWh per room
            else:
                home_state["rooms"][room_or_floor]["lights"] = value
                if value == "on":
                    home_state["usage"]["energy"] += 0.1
        elif device == "tv":
            home_state["rooms"][room_or_floor]["tv"] = value
            if value == "on":
                home_state["usage"]["energy"] += 0.2
        elif device == "electronics":
            if room_or_floor == "all":
                for room in home_state["rooms"]:
                    home_state["rooms"][room]["electronics"] = value
                    if value == "on":
                        home_state["usage"]["energy"] += 0.15  # 0.15 kWh per room
            else:
                home_state["rooms"][room_or_floor]["electronics"] = value
                if value == "on":
                    home_state["usage"]["energy"] += 0.15
        elif device == "security":
            if "passcode" in action:
                if action["passcode"] == home_state["security"]["passcode"]:
                    home_state["security"]["status"] = value
            else:
                home_state["security"]["status"] = value
        elif device == "hvac":
            home_state["floors"][room_or_floor]["temp"] = value
            home_state["usage"]["energy"] += 1.0
            home_state["usage"]["water"] += 0.5
        elif device == "lock":
            if "doors" in room_or_floor:
                for room in home_state["rooms"]:
                    home_state["rooms"][room]["lock"] = value
            elif "windows" in room_or_floor:
                for floor in home_state["floors"]:
                    home_state["floors"][floor]["windows"] = value
            else:
                home_state["rooms"][room_or_floor]["lock"] = value
    
    return home_state

# Core Function: Processes user command or query via LLM and updates home state or returns status
# Purpose: Interprets natural language input to control devices or query state, returning feedback
# Usage: Called per chat turn to parse command/query, update state if action, or return status if query
def process_command(command: str, home_state: dict) -> tuple[str, dict]:
    # Build prompt as a multi-line string, allowing actions or status queries
    prompt_parts = [
        "You are a home automation assistant controlling a simulated smart home. The current state is:\n",
        json.dumps(home_state, indent=2),
        f'\n\nParse the command or query: "{command}". Return *only* a JSON object:\n',
        '- For actions:\n',
        '  - Single: {"device": "lights", "location": "kitchen", "value": "on"}\n',
        '  - Multiple: {"actions": [{"device": "lights", "location": "all", "value": "off"}, ...]}\n',
        '- For status queries (e.g., "Are the lights on?"):\n',
        '  - {"query": true, "device": "lights", "location": "kitchen", "response": "Yes/No"}\n',
        'Include for actions:\n',
        '- "device": (e.g., "lights", "tv", "security", "hvac", "lock")\n',
        '- "location": (e.g., "living_room", "2nd", "all doors", "all" for entire house)\n',
        '- "value": (e.g., "on", "off", 72 for temp, "locked")\n',
        '- "passcode": (if security, e.g., "1234")\n',
        'Always use "location": "all" for commands affecting the entire house, even when turning devices on. If invalid, return {"error": "reason"}. Output must be a valid JSON string, nothing else.'
    ]
    prompt = "".join(prompt_parts)
    print(f"Generated prompt: {prompt[:100]}...")  # Debug to inspect prompt
    response = llm.invoke([SystemMessage(content=prompt), HumanMessage(content=command)])
    cleaned_response = response.content.strip().replace('```json', '').replace('```', '').strip()  # Remove Markdown fences
    print(f"Cleaned LLM response: '{cleaned_response}'")  # Debug to inspect cleaned response
    try:
        action_data = json.loads(cleaned_response)
        if "error" in action_data:
            return f"Error: {action_data['error']}", home_state
        if action_data.get("query", False):
            return action_data["response"], home_state  # Return status response without updating state
        new_state = update_home_state(home_state, action_data)
        if "actions" in action_data:
            actions_desc = ", ".join(f"{a['device']} in {a.get('location', 'home')} to {a['value']}" for a in action_data["actions"])
            return f"Actions taken: {actions_desc}. State updated.", new_state
        return f"Action taken: {action_data['device']} in {action_data.get('location', 'home')} set to {action_data['value']}. State updated.", new_state
    except Exception as e:
        return f"Error: Could not process command ({str(e)})", home_state

# Core Function: Runs the interactive home automation chat
# Purpose: Manages chat loop, processes commands/queries, updates state, and displays results
# Usage: Main entry point; runs until "exit," then shows final state and usage
def run_home_automation_chat():
    home_state = initialize_home_state()
    print("Welcome to the Home Automation Simulator! Control your virtual home (type 'exit' to quit):")
    while True:
        command = input("> ")
        if command.lower() == "exit":
            print("Goodbye!")
            print(f"\nFinal Home State:\n{json.dumps(home_state, indent=2)}")
            print(f"Total Energy Usage: {home_state['usage']['energy']} kWh")
            print(f"Total Water Usage: {home_state['usage']['water']} gal")
            break
        response, home_state = process_command(command, home_state)
        print(f"{response}\nCurrent State:\n{json.dumps(home_state, indent=2)}\n")

# Start the simulation
run_home_automation_chat()